In [2]:
import pandas as pd
import numpy as np

import sklearn
#import ensemble
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,mean_squared_error,balanced_accuracy_score,recall_score
import random
import lightgbm as lgb
import catboost as cbt

In [3]:
# data = pd.read_parquet("Data/Training/Amex_Campus_Challenge_Train.parquet")
data = pd.read_csv("Data/Training/bilkulfirstaayenge.csv")

In [4]:
# randomlist = random.sample(list(data.customer.unique()),int(0.1*len(data.customer.unique())))

In [5]:
# testdata = data[data['customer'].isin(randomlist)]

In [4]:
#select rows whose column value is not in a list
traindata = data
# traindata = data[~data['customer'].isin(randomlist)]

In [5]:
datarec = traindata[traindata['ind_recommended']==1].drop(['ind_recommended'], axis=1)
datanotrec = traindata[traindata['ind_recommended']==0].drop(['ind_recommended'], axis=1)

In [6]:
del data

In [7]:
def confusion_matrix_2col(df, col1, col2):
    return df.groupby([col1, col2]).size().unstack(fill_value=0)

confusion_matrix_2col(traindata, 'ind_recommended', 'activation')

activation,0,1
ind_recommended,,
0,292879,60173
1,80162,9843


In [8]:
def get_model(df):
    X_train, X_test, y_train, y_test = train_test_split(
        df.drop(['customer','merchant','activation'],axis=1), df['activation'], test_size=0.1, random_state=1)
    rec = cbt.CatBoostClassifier()
    rec.fit(X_train, y_train)
    y_pred = rec.predict(X_test)
    print("Balanced Accuracy :",balanced_accuracy_score(y_test, y_pred)," recall :",recall_score(y_test, y_pred))
    # print("mse : ",mean_squared_error(y_test, y_pred))
    return rec

In [9]:
datarec['customer'].groupby(datarec['activation']).count()

activation
0    80162
1     9843
Name: customer, dtype: int64

In [10]:
rec = get_model(datarec.drop(datarec[datarec['activation'] == 0].sample(n=60162).index))

Learning rate set to 0.04199
0:	learn: 0.6716800	total: 72.5ms	remaining: 1m 12s
1:	learn: 0.6536069	total: 79.1ms	remaining: 39.5s
2:	learn: 0.6381394	total: 86.1ms	remaining: 28.6s
3:	learn: 0.6224275	total: 92.8ms	remaining: 23.1s
4:	learn: 0.6094593	total: 99.3ms	remaining: 19.8s
5:	learn: 0.5963705	total: 106ms	remaining: 17.5s
6:	learn: 0.5852307	total: 112ms	remaining: 15.9s
7:	learn: 0.5741384	total: 118ms	remaining: 14.6s
8:	learn: 0.5647538	total: 123ms	remaining: 13.5s
9:	learn: 0.5564537	total: 130ms	remaining: 12.8s
10:	learn: 0.5500155	total: 136ms	remaining: 12.2s
11:	learn: 0.5427177	total: 142ms	remaining: 11.7s
12:	learn: 0.5367519	total: 148ms	remaining: 11.2s
13:	learn: 0.5302407	total: 154ms	remaining: 10.9s
14:	learn: 0.5244425	total: 161ms	remaining: 10.5s
15:	learn: 0.5193406	total: 167ms	remaining: 10.3s
16:	learn: 0.5142016	total: 173ms	remaining: 10s
17:	learn: 0.5103405	total: 180ms	remaining: 9.8s
18:	learn: 0.5064216	total: 187ms	remaining: 9.63s
19:	learn

In [11]:
datanotrec['customer'].groupby(datanotrec['activation']).count()

activation
0    292879
1     60173
Name: customer, dtype: int64

In [12]:
nrec = get_model(datanotrec.drop(datanotrec[datanotrec['activation'] == 0].sample(n=192879).index))

Learning rate set to 0.086051
0:	learn: 0.6541335	total: 23.9ms	remaining: 23.9s
1:	learn: 0.6217117	total: 40.9ms	remaining: 20.4s
2:	learn: 0.5970007	total: 57.4ms	remaining: 19.1s
3:	learn: 0.5768709	total: 73.4ms	remaining: 18.3s
4:	learn: 0.5588699	total: 89.7ms	remaining: 17.9s
5:	learn: 0.5452149	total: 107ms	remaining: 17.7s
6:	learn: 0.5347035	total: 123ms	remaining: 17.5s
7:	learn: 0.5248662	total: 140ms	remaining: 17.3s
8:	learn: 0.5160936	total: 157ms	remaining: 17.2s
9:	learn: 0.5093276	total: 174ms	remaining: 17.2s
10:	learn: 0.5025550	total: 192ms	remaining: 17.3s
11:	learn: 0.4964575	total: 211ms	remaining: 17.4s
12:	learn: 0.4921735	total: 233ms	remaining: 17.7s
13:	learn: 0.4876739	total: 259ms	remaining: 18.2s
14:	learn: 0.4842758	total: 278ms	remaining: 18.2s
15:	learn: 0.4810141	total: 294ms	remaining: 18.1s
16:	learn: 0.4783971	total: 311ms	remaining: 18s
17:	learn: 0.4754766	total: 328ms	remaining: 17.9s
18:	learn: 0.4727208	total: 345ms	remaining: 17.8s
19:	lear

In [13]:
name = "AAC_newdata"

In [14]:
#save models

import os
os.makedirs('submissions/'+name+'/models', exist_ok=True)

In [15]:
import joblib

In [16]:
joblib.dump(rec, 'submissions/'+name+'/models/recact.joblib')
joblib.dump(nrec, 'submissions/'+name+'/models/nrecact.joblib')

['submissions/AAC_newdata/models/nrecact.joblib']